In [15]:
import tensorflow_datasets as tfds
import jax.numpy as jnp
import jax
from jax import random, jit, grad

In [16]:
# MNIST 데이터 불러오기
ds = tfds.load('mnist', split='train', as_supervised=True)

In [9]:
# 데이터 전처리 함수
def preprocess(image, label):
    image = jnp.array(image, dtype=jnp.float32) / 255.0
    label = jnp.array(label, dtype=jnp.int32)
    return image.reshape(-1), label

# 데이터 변환
train_data = [(preprocess(image, label)) for image, label in tfds.as_numpy(ds)]
print(f'훈련 데이터 샘플 수: {len(train_data)}')

훈련 데이터 샘플 수: 60000


In [10]:
# 신경망 모델 정의

## 파라미터 초기화 함수
def init_params(layer_sizes, key):
    params = []
    for n_in, n_out in zip(layer_sizes[:-1], layer_sizes[1:]):
        key, subkey = random.split(key)
        weights = random.normal(subkey, (n_in, n_out)) * 0.01
        biases = jnp.zeros(n_out)
        params.append((weights, biases))
    return params

## MLP 모델 함수
def predict(params, x):
    for w, b in params[:-1]:
        x = jnp.dot(x, w) + b
        x = jnp.maximum(x, 0) # ReLu 활성화 함수
    final_w, final_b = params[-1]
    logits = jnp.dot(x, final_w) + final_b
    return logits - jax.scipy.special.logsumexp(logits, axis=1, keepdims=True)

In [11]:
# 손실함수 및 정확도 계산

## 손실함수
def cross_entropy_loss(params, x, y):
    logits = predict(params, x)
    one_hot = jax.nn.one_hot(y, num_classes=10)
    return -jnp.mean(jnp.sum(one_hot * logits, axis=1))

## 정확도 계산 함수
def accuracy(params, x, y):
    logits = predict(params, x)
    predictions = jnp.argmax(logits, axis=1)
    return jnp.mean(predictions == y)

In [12]:
# 학습 루프 정의

## 훈련단계
learning_rate = 0.01
epochs = 5
batch_size = 128
key = random.PRNGKey(42)

# 모델 초기화
params = init_params([784, 128, 10], key)

# 기울기 계산 함수
grad_loss = jit(grad(cross_entropy_loss))

# 파라미터 업데이트 함수
@jit
def update(params, x, y, lr):
    grads = grad_loss(params, x, y)
    return [(w - lr * dw, b - lr * db) for (w, b), (dw, db) in zip(params, grads)]

In [13]:
# 모델 학습

for epoch in range(epochs):
    # 미니배치 학습
    for i in range(0, len(train_data), batch_size):
        batch = train_data[i:i + batch_size]
        x_batch, y_batch = zip(*batch)
        x_batch = jnp.stack(x_batch)
        y_batch = jnp.array(y_batch)

        # 파라미터 업데이트
        params = update(params, x_batch, y_batch, learning_rate)

    # 에포크별 손실 및 정확도 출력
    train_loss = cross_entropy_loss(params, x_batch, y_batch)
    train_acc = accuracy(params, x_batch, y_batch)
    print(f"Epoch {epoch + 1}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

Epoch 1, Loss: 2.0188, Accuracy: 0.6354
Epoch 2, Loss: 0.9221, Accuracy: 0.8125
Epoch 3, Loss: 0.5631, Accuracy: 0.8854
Epoch 4, Loss: 0.4253, Accuracy: 0.9167
Epoch 5, Loss: 0.3521, Accuracy: 0.9479


In [14]:
# 학습 결과 평가

# 테스트 데이터 로드
ds_test = tfds.load('mnist', split='test', as_supervised=True)
test_data = [(preprocess(image, label)) for image, label in tfds.as_numpy(ds_test)]

# 평가
x_test, y_test = zip(*test_data)
x_test = jnp.stack(x_test)
y_test = jnp.array(y_test)

test_acc = accuracy(params, x_test, y_test)
print(f"테스트 정확도: {test_acc:.4f}")

테스트 정확도: 0.8828
